Name    : Fathi Al Adha Hylmi
NIM     : 22/492195/PA/21088

In [1]:
import pandas as pd
import nltk
import re
from collections import defaultdict
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hylmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hylmi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Download NLTK data if not already installed
nltk.download('punkt')

# Load the dataset
df = pd.read_csv("C:/Users/hylmi/OneDrive - UGM 365/Semester 5/Information Retrieval/News.csv")
df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hylmi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,id_author,title,portal,time,author,editor,content,source
0,0,1,Infografis Pekerja Asing Dilarang Masuk Wilaya...,Liputan6.com,"24 Jul 2021, 09:02 WIB",Abdillah,Abdillah,Pemerintah melalui Menteri Hukum dan Hak Asasi...,https://www.liputan6.com/news/read/4614451/inf...
1,1,1,Infografis Jadwal Bulu Tangkis Indonesia di Ol...,Liputan6.com,"23 Jul 2021, 23:23 WIB",Abdillah,Abdillah,Bulu Tangkis menjadi andalan Indonesia berburu...,https://www.liputan6.com/bola/read/4614427/inf...
2,2,1,"Infografis Jangan Bebal, Kamu Tidak Kebal Covi...",Liputan6.com,"23 Jul 2021, 10:40 WIB",Abdillah,Abdillah,Covid-19 tidak mengenal usia dan status. Siapa...,https://www.liputan6.com/news/read/4613233/inf...
3,3,1,Infografis Awas Perokok Lebih Rentan Tertular ...,Liputan6.com,"22 Jul 2021, 10:35 WIB",Abdillah,Abdillah,Kebiasaan merokok berisiko menimbulkan sejumla...,https://www.liputan6.com/news/read/4612324/inf...
4,4,1,Infografis Perbedaan Aturan PPKM Level 3 dan 4,Liputan6.com,"22 Jul 2021, 09:01 WIB",Abdillah,Abdillah,Pemberlakuan Pembatasan Kegiatan Masyarakat at...,https://www.liputan6.com/news/read/4612511/inf...
...,...,...,...,...,...,...,...,...,...
14338,14339,88,Berhemat Ruang Isolasi Pasien Covid,Merdeka.com,"Senin, 7 September 2020 09:14",Wilfridus Setu Embu,-,Kasus baru pasien positif Covid-19 masih bel...,https://www.merdeka.com/khas/berhemat-ruang-is...
14339,14340,88,Kendala Memadamkan Kebakaran Gedung Kejagung,Merdeka.com,"Senin, 31 Agustus 2020 10:37",Wilfridus Setu Embu,-,Laporan kebakaran datang pukul 19.10 WIB. Ti...,https://www.merdeka.com/khas/kendala-memadamka...
14340,14341,88,Kobaran Api Kejagung RI,Merdeka.com,"Senin, 31 Agustus 2020 09:35",Wilfridus Setu Embu,-,"Menjelang petang, Marni sedang sibuk melayan...",https://www.merdeka.com/khas/kobaran-api-kejag...
14341,14342,88,"Kampus Ruang Diskusi, Pendidikan Militer Belum...",Merdeka.com,"Senin, 24 Agustus 2020 10:06",Wilfridus Setu Embu,-,Banyak cara wewujudkan rasa nasionalisme kau...,https://www.merdeka.com/khas/kampus-ruang-disk...


In [3]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    return text

df['clean_punct'] = df['content'].dropna().apply(clean_punct)
df['clean_punct']

0        Pemerintah melalui Menteri Hukum dan Hak Asasi...
1        Bulu Tangkis menjadi andalan Indonesia berburu...
2        Covid-19 tidak mengenal usia dan status. Siapa...
3        Kebiasaan merokok berisiko menimbulkan sejumla...
4        Pemberlakuan Pembatasan Kegiatan Masyarakat at...
                               ...                        
14338      Kasus baru pasien positif Covid-19 masih bel...
14339      Laporan kebakaran datang pukul 19.10 WIB. Ti...
14340      Menjelang petang Marni sedang sibuk melayani...
14341      Banyak cara wewujudkan rasa nasionalisme kau...
14342      Isu pendidikan ala militer masuk kampus buru...
Name: clean_punct, Length: 14343, dtype: object

In [4]:
# prompt: lower case everything in a column
df['lower'] = df['clean_punct'].dropna().apply(lambda x: x.lower())
df['lower']

0        pemerintah melalui menteri hukum dan hak asasi...
1        bulu tangkis menjadi andalan indonesia berburu...
2        covid-19 tidak mengenal usia dan status. siapa...
3        kebiasaan merokok berisiko menimbulkan sejumla...
4        pemberlakuan pembatasan kegiatan masyarakat at...
                               ...                        
14338      kasus baru pasien positif covid-19 masih bel...
14339      laporan kebakaran datang pukul 19.10 wib. ti...
14340      menjelang petang marni sedang sibuk melayani...
14341      banyak cara wewujudkan rasa nasionalisme kau...
14342      isu pendidikan ala militer masuk kampus buru...
Name: lower, Length: 14343, dtype: object

In [5]:
#remove double or more whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")
df['clean_content'] = df['lower'].dropna().apply(_normalize_whitespace)
df['clean_content']

0        pemerintah melalui menteri hukum dan hak asasi...
1        bulu tangkis menjadi andalan indonesia berburu...
2        covid-19 tidak mengenal usia dan status. siapa...
3        kebiasaan merokok berisiko menimbulkan sejumla...
4        pemberlakuan pembatasan kegiatan masyarakat at...
                               ...                        
14338    kasus baru pasien positif covid-19 masih belum...
14339    laporan kebakaran datang pukul 19.10 wib. tim ...
14340    menjelang petang marni sedang sibuk melayani p...
14341    banyak cara wewujudkan rasa nasionalisme kaum ...
14342    isu pendidikan ala militer masuk kampus buru-b...
Name: clean_content, Length: 14343, dtype: object

In [6]:
# Calculate the number of documents
num_docs = df['content'].notnull().sum()

# Function to count words/terms
def count_words(text):
    words = nltk.word_tokenize(text)
    return len(words)

# Function to count sentences
def count_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

# Initialize counts
total_words = 0
total_sentences = 0

# Iterate through the dataset and calculate words and sentences
for content in df['clean_content'].dropna():
    total_words += count_words(content)
    total_sentences += count_sentences(content)

# Display the results
print(f"Total Documents: {num_docs}")
print(f"Total Words: {total_words}")
print(f"Total Sentences: {total_sentences}")

Total Documents: 14334
Total Words: 5987490
Total Sentences: 322544


In [7]:
# Timing the Inverted Index Creation
start_time_index = time.time()

# Initialize an inverted index using defaultdict
inverted_index = defaultdict(list)

# Build the inverted index
for index, row in df.iterrows():
    doc_id = row['id']
    content = row['clean_content']
    
    if pd.notnull(content):
        words = nltk.word_tokenize(content.lower())
        for word in set(words):  # Use set to avoid duplicate document entries
            inverted_index[word].append(doc_id)

# Convert defaultdict to a regular dictionary
inverted_index = dict(inverted_index)

# End timing for inverted index creation
end_time_index = time.time()
index_creation_time = end_time_index - start_time_index

In [8]:
print(f"Inverted Index Creation Time: {index_creation_time:.4f} seconds\n")

Inverted Index Creation Time: 27.1853 seconds



In [9]:
# Function to get document frequency for a term
def doc_frequency(term, inverted_index):
    return len(inverted_index.get(term, []))

# Function for boolean retrieval with sorting by term frequency and returning document count
def boolean_retrieval(query, inverted_index, total_docs):
    start_time_query = time.time()
    
    terms = query.lower().split()  # Tokenize the query
    logical_operator = None
    
    # Determine logical operator and identify terms
    if "and" in terms:
        logical_operator = "and"
        term1, term2 = terms[0], terms[2]
    elif "or" in terms:
        logical_operator = "or"
        term1, term2 = terms[0], terms[2]
    elif "not" in terms:
        logical_operator = "not"
        term1, term2 = terms[0], terms[2]
    else:
        # Single term query
        term1 = terms[0]
        result_set = set(inverted_index.get(term1, []))
        end_time_query = time.time()
        query_time = end_time_query - start_time_query
        return result_set, len(result_set), query_time

    # Sort terms by document frequency (smallest first)
    sorted_terms = sorted([term1, term2], key=lambda term: doc_frequency(term, inverted_index))

    # Perform AND operation
    if logical_operator == "and":
        if sorted_terms[0] in inverted_index and sorted_terms[1] in inverted_index:
            result_set = set(inverted_index[sorted_terms[0]]) & set(inverted_index[sorted_terms[1]])  # Intersection
        else:
            result_set = set()  # Return empty set if any term not found

    # Perform OR operation
    elif logical_operator == "or":
        docs_term1 = set(inverted_index.get(sorted_terms[0], []))
        docs_term2 = set(inverted_index.get(sorted_terms[1], []))
        result_set = docs_term1 | docs_term2  # Union

    # Perform NOT operation
    elif logical_operator == "not":
        if sorted_terms[0] in inverted_index:
            result_set = set(inverted_index[sorted_terms[0]]) - set(inverted_index.get(sorted_terms[1], []))  # Difference
        else:
            result_set = set()  # Return empty set if the first term not found
    
    end_time_query = time.time()
    query_time = end_time_query - start_time_query
    
    return result_set, len(result_set), query_time

In [10]:
query1 = "masyarakat and pemberlakuan not bisa"
# Perform Boolean Retrieval
result1, count1, time1 = boolean_retrieval(query1, inverted_index, total_docs=len(df))
print(f"Results for '{query1}': Document Count = {count1}, Documents = {result1}, Time Taken = {time1:.4f} seconds")

Results for 'masyarakat and pemberlakuan not bisa': Document Count = 1187, Documents = {0, 2049, 14336, 10243, 4, 2052, 6, 2053, 8, 9, 10, 12620, 12621, 6154, 6155, 6158, 6160, 12622, 6167, 6168, 2076, 6173, 32, 10272, 2082, 2083, 2084, 10277, 2086, 6176, 2088, 6178, 10282, 10283, 6180, 10285, 6184, 2096, 10291, 6195, 6201, 10301, 10303, 6208, 4162, 4164, 4166, 6215, 6220, 77, 80, 81, 82, 2131, 4178, 85, 4179, 4181, 6225, 6230, 6231, 6233, 2140, 2141, 6235, 6236, 6237, 6238, 6240, 10339, 6241, 6242, 2150, 13860, 6244, 6246, 2155, 10349, 111, 2159, 113, 2161, 2162, 116, 117, 2163, 6260, 128, 133, 2187, 6283, 2189, 6284, 6285, 12432, 6288, 2199, 6295, 6297, 2209, 6313, 6314, 6316, 6317, 6319, 178, 6322, 2228, 6323, 2230, 183, 6324, 2233, 6325, 10427, 6327, 6328, 6331, 6333, 6334, 6335, 6336, 6337, 6340, 6341, 6342, 6343, 6344, 6345, 6146, 6347, 6348, 6349, 2255, 6350, 2257, 2270, 4319, 6369, 234, 240, 245, 4351, 4355, 10500, 8454, 2310, 2311, 8458, 4365, 4372, 4374, 8472, 4377, 10524, 43

In [11]:
query2 = "bisa or pada and negara"
# Perform Boolean Retrieval
result2, count2, time2 = boolean_retrieval(query2, inverted_index, total_docs=len(df))
print(f"Results for '{query2}': Document Count = {count2}, Documents = {result2}, Time Taken = {time2:.4f} seconds")

Results for 'bisa or pada and negara': Document Count = 5006, Documents = {0, 6, 12, 13, 14, 15, 18, 19, 22, 23, 25, 26, 29, 30, 31, 35, 38, 39, 40, 41, 42, 44, 45, 47, 48, 49, 51, 54, 55, 56, 57, 61, 62, 63, 64, 66, 67, 71, 73, 75, 78, 80, 81, 82, 85, 86, 87, 88, 92, 96, 97, 99, 100, 101, 103, 104, 105, 108, 109, 110, 111, 112, 114, 119, 123, 125, 126, 127, 129, 130, 133, 138, 140, 143, 146, 148, 149, 151, 153, 154, 157, 158, 161, 164, 167, 168, 169, 171, 172, 181, 182, 185, 190, 191, 192, 193, 194, 195, 197, 198, 199, 202, 203, 204, 206, 207, 208, 209, 211, 212, 213, 214, 215, 216, 218, 229, 244, 245, 258, 259, 261, 264, 265, 267, 273, 274, 277, 279, 280, 281, 283, 288, 290, 294, 296, 297, 298, 302, 303, 305, 306, 310, 311, 313, 314, 315, 317, 318, 320, 321, 322, 325, 327, 328, 336, 338, 340, 342, 343, 344, 345, 346, 347, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 361, 362, 363, 364, 366, 367, 368, 369, 370, 371, 373, 374, 376, 377, 378, 380, 381, 382, 385, 386, 387, 388, 389,

In [12]:
query3 = "rabu and warga"
# Perform Boolean Retrieval
result3, count3, time3 = boolean_retrieval(query3, inverted_index, total_docs=len(df))
print(f"Results for '{query3}': Document Count = {count3}, Documents = {result3}, Time Taken = {time3:.4f} seconds")

Results for 'rabu and warga': Document Count = 697, Documents = {0, 14343, 11, 8205, 10255, 8208, 12307, 10261, 12310, 12311, 8218, 4123, 8220, 10273, 10274, 10275, 12323, 10277, 4132, 12328, 12330, 2098, 8251, 12348, 2109, 12350, 8255, 12351, 12352, 8259, 2117, 10311, 2120, 10312, 75, 2123, 12366, 12368, 2129, 84, 12372, 2137, 12379, 8285, 8286, 12383, 2144, 8289, 8290, 12385, 10344, 10345, 10347, 10349, 10350, 111, 8309, 119, 12413, 8322, 8323, 8325, 8326, 2185, 12426, 10379, 10385, 10386, 10387, 10388, 157, 160, 8354, 8356, 8357, 10406, 8361, 8364, 12466, 12467, 10420, 10421, 6143, 8383, 8384, 2241, 2243, 2244, 8389, 12486, 12491, 12497, 12498, 10451, 12502, 12504, 6361, 8410, 8414, 8419, 10467, 10468, 10469, 8423, 8426, 8427, 12526, 243, 10483, 10484, 10489, 10491, 10493, 6406, 10511, 10512, 10513, 4373, 12566, 4375, 12570, 4379, 4382, 4383, 10534, 2357, 6456, 2363, 6459, 12608, 8514, 4420, 8517, 8522, 8525, 12624, 12643, 2404, 10597, 6502, 6506, 2413, 6510, 6511, 2417, 2419, 10638

In [13]:
query4 = "hukum and sosialisasi not pemberlakuan"
# Perform Boolean Retrieval
result4, count4, time4 = boolean_retrieval(query4, inverted_index, total_docs=len(df))
print(f"Results for '{query4}': Document Count = {count4}, Documents = {result4}, Time Taken = {time4:.4f} seconds")

Results for 'hukum and sosialisasi not pemberlakuan': Document Count = 63, Documents = {0, 14336, 9728, 3203, 3842, 389, 8454, 10502, 8584, 8588, 9486, 8465, 4375, 11418, 1051, 4381, 3741, 8094, 13600, 11680, 14241, 10277, 2086, 10917, 168, 8874, 12075, 9908, 2357, 10295, 697, 11963, 6716, 4924, 11964, 1343, 11965, 3009, 11966, 11203, 11968, 7882, 1237, 12891, 3677, 7650, 4066, 10346, 8939, 10860, 6637, 6638, 5229, 11628, 9706, 9712, 11884, 5364, 12141, 14319, 8059, 5372, 2814}, Time Taken = 0.0000 seconds


In [14]:
query5 = "orang or warga and sosialisasi"
# Perform Boolean Retrieval
result5, count5, time5 = boolean_retrieval(query5, inverted_index, total_docs=len(df))
print(f"Results for '{query5}': Document Count = {count5}, Documents = {result5}, Time Taken = {time5:.4f} seconds")

Results for 'orang or warga and sosialisasi': Document Count = 1620, Documents = {0, 8201, 8202, 11, 8206, 8207, 8208, 8215, 8222, 8224, 8226, 8229, 8232, 8233, 8243, 8244, 8246, 8255, 8260, 8261, 8266, 75, 8267, 8268, 8270, 8272, 8275, 84, 8277, 8285, 8286, 8290, 8291, 8292, 8295, 8302, 111, 8309, 8311, 119, 8312, 128, 8322, 131, 8323, 8328, 137, 8333, 143, 8341, 8342, 8343, 155, 157, 160, 8353, 8354, 8356, 8357, 8364, 8366, 8369, 8370, 8374, 8377, 8382, 8384, 8389, 204, 205, 8396, 8399, 215, 8410, 8411, 8413, 8418, 8419, 8423, 8426, 8427, 8435, 8437, 8442, 8445, 8446, 8447, 8450, 8452, 8462, 8482, 8483, 8485, 8496, 8508, 8546, 372, 8564, 8565, 8566, 8569, 380, 384, 8579, 396, 405, 416, 8614, 439, 8638, 459, 461, 468, 8660, 8662, 8663, 8670, 8671, 487, 8694, 8698, 8744, 8745, 8803, 613, 635, 8831, 658, 676, 8874, 8884, 8887, 697, 8922, 8927, 8952, 782, 791, 797, 818, 846, 851, 860, 861, 9053, 9054, 866, 9059, 873, 9065, 876, 9068, 881, 9073, 9080, 9090, 9091, 903, 909, 915, 9111, 9113